## Bulk Copy

In [ ]:
import pandas as pd
import altair as alt

alt.theme.enable("latimes")

df = pd.read_csv(
    "Benchmark.BulkCopyBenchmarks-report.csv",
    delimiter=",",
)


def clean_data(df):
    # Clean numeric fields from strings like "49.51 ms"
    df["Median"] = df["Median"].str.replace(" ms", "", case=False, regex=False)
    df["Median"] = df["Median"].str.replace(",", "", case=False, regex=False)
    df = df.astype({"Median": "float64"})

    # Map labels
    mapping = {
        "NoIndex": "no index",
        "Res": "resource_id",
        "Res_BillDate": "resource_id, billing_date",
        "Res_BillDate_Cst": ["resource_id, billing_date,", "cost"],
        "Res_BillDate_Inc_Cst": ["resource_id, billing_date", "include cost"],
        "After": ["resource_id, billing_date", "include cost - after insert"],
    }
    df["index_label"] = df["IndexCreation"].map(mapping).fillna(df["IndexCreation"])

    return df


df_clean = clean_data(df.copy())

# Main chart: Execution time
main = (
    alt.Chart(df_clean)
    .mark_line()
    .encode(
        y=alt.Y("Median", sort="x")
        .scale(type="log")
        .title("Median bulk copy time (ms)"),
        x=alt.X("BatchSize").scale(type="log").title("Batch size (rows)"),
        color=alt.Color(
            "index_label",
            title="Index creation strategy",
            scale=alt.Scale(scheme="tableau10"),
        ),
    )
    .properties(
        title="PostgreSQL bulk copy performance by different indexing strategies",
        height=240,
        width=640,
    )
)

# Relative chart: Relative to no index (%)
baseline = df_clean[df_clean["index_label"] == "no index"][["BatchSize", "Median"]].set_index("BatchSize")
df_clean["relative"] = df_clean.apply(
    lambda row: (row["Median"] / baseline.loc[row["BatchSize"], "Median"] * 100)
    if row["BatchSize"] in baseline.index else None,
    axis=1
)

relative_plot = (
    alt.Chart(df_clean)
    .mark_line()
    .encode(
        y=alt.Y("relative").title("Relative to 'no index' (%)"),
        x=alt.X("BatchSize").scale(type="log").title("Batch size (rows)"),
        color=alt.Color(
            "index_label",
            title="Index creation strategy",
            scale=alt.Scale(scheme="tableau10"),
        ),
    )
    .properties(
        height=240,
        width=640,
    )
)

# Combine plots vertically
combined = main & relative_plot

# Render and save
combined.show()
combined.save("results.png", scale_factor=2)

alt.VConcatChart(...)

## Bulk Copy per table creation strategy

In [ ]:
import pandas as pd
import altair as alt

alt.theme.enable("latimes")

df = pd.read_csv(
    "Benchmark.BulkCopyPerTableCreation-report.csv",
    delimiter=",",
)


def clean_data(df):
    # Clean numeric fields from strings like "49.51 ms"
    df["Mean"] = df["Mean"].str.replace(" ms", "", case=False, regex=False)
    df["Mean"] = df["Mean"].str.replace(",", "", case=False, regex=False)
    df = df.astype({"Mean": "float64"})

    return df


df_clean = clean_data(df.copy())

# Main chart: Execution time
main = (
    alt.Chart(df_clean[df_clean["CreateIndex"] == "with index"])
    .mark_line(point=True)
    .encode(
        y=alt.Y("Mean", sort="x").scale(type="log").title("Mean bulk copy time (ms)"),
        x=alt.X("BatchSize").scale(type="log").title("Batch size (rows)"),
        color=alt.Color(
            "CreateTable",
            title="Table creation strategy",
            scale=alt.Scale(scheme="tableau10"),
        ),
    )
    .properties(
        title=alt.Title(
            text="PostgreSQL bulk copy performance by batch size",
            subtitle="Tables with index",
        ),
        height=640,
        width=640,
    )
)

bottom = (
    alt.Chart(df_clean[df_clean["CreateIndex"] == "no index"])
    .mark_line(point=True)
    .encode(
        y=alt.Y("Mean", sort="x").scale(type="log").title("Mean bulk copy time (ms)"),
        x=alt.X("BatchSize").scale(type="log").title("Batch size (rows)"),
        color=alt.Color(
            "CreateTable",
            title="Table creation strategy",
            scale=alt.Scale(scheme="tableau10"),
        ),
    )
    .properties(
        title=alt.Title(
            text="PostgreSQL bulk copy performance by batch size",
            subtitle="Tables without index",
        ),
        height=640,
        width=640,
    )
)

# Render and save
main.show()
main.save("bulk_copy_by_table_type_with_index.png", scale_factor=2)

bottom.show()
bottom.save("bulk_copy_by_table_type_no_index.png", scale_factor=2)

alt.Chart(...)

alt.Chart(...)

In [ ]:
import pandas as pd
import altair as alt

alt.theme.enable("latimes")

df = pd.read_csv(
    "Benchmark.TempBuffeInfluence-report.csv",
    delimiter=",",
)


def clean_data(df):
    # Clean numeric fields from strings like "49.51 ms"
    df["Mean"] = df["Mean"].str.replace(" s", "", case=False, regex=False)
    df["Mean"] = df["Mean"].str.replace(",", "", case=False, regex=False)
    df = df.astype({"Mean": "float64"})

    return df


df_clean = clean_data(df.copy())

# Main chart: Execution time
main = (
    alt.Chart(df_clean)
    .mark_line(point=True)
    .encode(
        y=alt.Y("Mean", sort="x").title("Mean bulk copy time (ms)"),
        x=alt.X("TempBufferSize").scale(type="log").title("Temp Buffer Size (8 kB units)"),
        color=alt.Color(
            "CreateTable",
            title="Table creation strategy",
            scale=alt.Scale(scheme="tableau10"),
        ),
    )
    .properties(
        title=alt.Title(
            text="PostgreSQL bulk copy performance by temp buffer size",
            subtitle="Tables with index",
        ),
        height=640,
        width=640,
    )
)

# Render and save
main.show()
main.save("temp_buffer_size_influence.png", scale_factor=2)

alt.Chart(...)

## All In One

In [10]:
import glob
import pandas as pd

# Get all .csv files
csv_files = glob.glob("../../*.csv")

def median(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    filtered = series[(series >= lower) & (series <= upper)]
    return filtered.median()

summary = []
for f in csv_files:
    try:
        df = pd.read_csv(f)
        # Measurements
        createTableTime = median(df["CreateTableTime"])
        createIndexTime = median(df["CreateIndexTime"])
        bulkCopyTime = median(df["BulkCopyTime"])
        analyzeTime = median(df["AnalyzeTime"])
        selectTime = median(df["SelectTime"])

        # Params
        createTable = df["CreateTable"][0]
        createIndex = df["CreateIndex"][0]
        analyze = df["Analyze"][0]
        workers = df["Workers"][0]
        minIndexScanSize = df["MinIndexScanSize"][0]
        minTableScanSize = df["MinTableScanSize"][0]
        setupCost = df["SetupCost"][0]
        tupleCost = df["TupleCost"][0]
        hashAgg = df["HashAgg"][0]

        # Execution plan
        executionPlan = df["ExecutionPlan"][0]

        summary.append({
            "createTableTime": createTableTime,
            "createIndexTime": createIndexTime,
            "bulkCopyTime": bulkCopyTime,
            "analyzeTime": analyzeTime,
            "selectTime": selectTime,
            "createTable": str(createTable).strip(),
            "createIndex": str(createIndex).strip(),
            "analyze": str(analyze).strip(),
            "workers": str(workers).strip(),
            "minIndexScanSize": str(minIndexScanSize).strip(),
            "minTableScanSize": str(minTableScanSize).strip(),
            "setupCost": str(setupCost).strip(),
            "tupleCost": str(tupleCost).strip(),
            "hashAgg": str(hashAgg).strip(),
            "executionPlan": str(executionPlan).strip(),
        })
    except Exception as e:
        print(f"Error loading {f}: {e}")

summary_df = pd.DataFrame(summary)

summary_df

,createTableTime,createIndexTime,bulkCopyTime,analyzeTime,selectTime,createTable,createIndex,analyze,workers,minIndexScanSize,minTableScanSize,setupCost,tupleCost,hashAgg,executionPlan
0,27.0,0.0,465.0,92.0,88.0,regular,no_index,analyze,4,0,0,0,0,on,Parallel HashAggregate Seq Scan
1,1.0,0.0,473.0,110.0,240.5,temp,no_index,analyze,2,64,1024,1000,0.1,on,HashAggregate Seq Scan
2,27.0,0.0,408.0,0.0,294.5,unlogged,no_index,no_analyze,4,0,0,0,0,off,Parallel GroupAggregate Seq Scan
3,1.0,1.0,6602.5,111.0,3011.0,temp,with_index,analyze,4,0,0,0,0,off,GroupAggregate Index Only Scan
4,24.0,24.0,2998.0,0.0,85.0,unlogged,with_index,no_analyze,4,0,0,0,0,on,Parallel HashAggregate Seq Scan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,1.0,0.0,426.0,0.0,1135.0,temp,no_index,no_analyze,4,0,0,0,0,off,GroupAggregate Seq Scan
68,0.0,0.0,525.0,93.0,457.0,regular,no_index,analyze,2,64,1024,1000,0.1,off,Parallel GroupAggregate Seq Scan
69,24.0,24.0,3679.0,95.0,106.0,regular,with_index,analyze,4,0,0,0,0,on,Parallel HashAggregate Seq Scan
70,26.0,0.0,494.0,0.0,293.0,regular,no_index,no_analyze,4,0,0,0,0,off,Parallel GroupAggregate Seq Scan


In [ ]:
import altair as alt

alt.theme.enable("latimes")

filtered = summary_df[
    (summary_df["createIndex"] == "no_index")
    & (summary_df["analyze"] == "analyze")
    & (summary_df["workers"] == "2")
    & (summary_df["hashAgg"] == "on")
].copy()

bar = (
    alt.Chart(filtered)
    .mark_bar()
    .encode(
        y=alt.Y("bulkCopyTime:Q").title("Median time (ms)"),
        x=alt.X("createTable:N").title(None),
        color=alt.Color(
            "createTable:N",
            title="Table type",
            scale=alt.Scale(scheme="tableau10"),
            legend=alt.Legend(
                orient="right",
                title="Table type",
                labelLimit=300,
            ),
        ),
    )
)

text = (
    alt.Chart(filtered)
    .mark_text(dy=12)
    .encode(
        y=alt.Y("bulkCopyTime:Q"),
        x=alt.X("createTable:N"),
        text=alt.Text("bulkCopyTime:Q", format=".1f"),
    )
)

main = (bar + text).properties(
    title=alt.Title(
        text="PostgreSQL bulk copy performance by different table types",
        subtitle="Tables without indexes",
    ),
    width=480,
    height=640,
)

main.save("copy_no_index.png", scale_factor=2)
main

alt.LayerChart(...)

In [ ]:
import altair as alt

alt.theme.enable("latimes")

filtered = summary_df[
    (summary_df["createIndex"] == "with_index")
    & (summary_df["analyze"] == "analyze")
    & (summary_df["workers"] == "2")
    & (summary_df["hashAgg"] == "on")
].copy()

bar = (
    alt.Chart(filtered)
    .mark_bar()
    .encode(
        y=alt.Y("bulkCopyTime:Q").title("Median time (ms)"),
        x=alt.X("createTable:N").title(None),
        color=alt.Color(
            "createTable:N",
            title="Table type",
            scale=alt.Scale(scheme="tableau10"),
            legend=alt.Legend(
                orient="right",
                title="Table type",
                labelLimit=300,
            ),
        ),
    )
)

text = (
    alt.Chart(filtered)
    .mark_text(dy=12)
    .encode(
        y=alt.Y("bulkCopyTime:Q"),
        x=alt.X("createTable:N"),
        text=alt.Text("bulkCopyTime:Q", format=".1f"),
    )
)

main = (bar + text).properties(
    title=alt.Title(
        text="PostgreSQL bulk copy performance by different table types",
        subtitle="Tables without indexes",
    ),
    width=480,
    height=640,
)

main.save("copy_with_index.png", scale_factor=2)
main

alt.LayerChart(...)

# Select (default settings)

In [45]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()
alt.theme.enable("latimes")

df = pd.read_csv(
    "./../../select_results.csv",
    delimiter=",",
)

df.columns = df.columns.str.strip()

# make sure numeric columns are numeric
df['SelectTime'] = pd.to_numeric(df['SelectTime'], errors='coerce')
df['BatchSize'] = pd.to_numeric(df['BatchSize'], errors='coerce')

# strip whitespace from string columns
df['CreateTable']   = df['CreateTable'].astype(str).str.strip()
df['ExecutionPlan'] = df['ExecutionPlan'].astype(str).str.strip().str.strip('"')

# group and compute average select time
grouped = (
    df
    .groupby(['CreateTable', 'BatchSize', 'ExecutionPlan'], dropna=False)['SelectTime']
    .mean()
    .reset_index(name='Mean')
)

# Main chart: Execution time
main = (
    alt.Chart(grouped)
    .mark_circle()
    .encode(
        y=alt.Y("Mean", sort="x").scale(type="log").title("Mean select time (ms)"),
        x=alt.X("BatchSize").scale(type="log").title("Batch size (rows)"),
        color=alt.Color(
            "CreateTable",
            title="Table creation strategy",
            scale=alt.Scale(scheme="tableau10"),
        ),
    )
    .properties(
        title=alt.Title(
            text="PostgreSQL SELECT performance by batch size",
        ),
        height=240,
        width=860,
    )
)

regular = (
    alt.Chart(grouped[grouped["CreateTable"] == 'regular'])
    .mark_circle()
    .encode(
        y=alt.Y("Mean", sort="x").scale(type="log").title("Mean select time (ms)"),
        x=alt.X("BatchSize").scale(type="log").title("Batch size (rows)"),
        color=alt.Color(
            "ExecutionPlan",
            title="Execution Plan",
            scale=alt.Scale(scheme="tableau10"),
        ),
    )
    .properties(
        title=alt.Title(
            text="SELECT from regular table",
            fontSize=14
        ),
        height=240,
        width=240,
    )
)

unlogged = (
    alt.Chart(grouped[grouped["CreateTable"] == 'unlogged'])
    .mark_circle()
    .encode(
        y=alt.Y("Mean", sort="x").scale(type="log").title("Mean select time (ms)"),
        x=alt.X("BatchSize").scale(type="log").title("Batch size (rows)"),
        color=alt.Color(
            "ExecutionPlan",
            title="Execution Plan",
            scale=alt.Scale(scheme="tableau10"),
        ),
    )
    .properties(
        title=alt.Title(
            text="SELECT from unlogged table",
            fontSize=14
        ),
        height=240,
        width=240,
    )
)

temp = (
    alt.Chart(grouped[grouped["CreateTable"] == 'temp'])
    .mark_circle()
    .encode(
        y=alt.Y("Mean", sort="x").scale(type="log").title("Mean select time (ms)"),
        x=alt.X("BatchSize").scale(type="log").title("Batch size (rows)"),
        color=alt.Color(
            "ExecutionPlan",
            title="Execution Plan",
            scale=alt.Scale(scheme="tableau10"),
        ),
    )
    .properties(
        title=alt.Title(
            text="SELECT from temp table",
            fontSize=14
        ),
        height=240,
        width=240,
    )
)

chart = main & (regular | unlogged | temp)

# Render and save
chart = chart.resolve_scale(
    color='independent'
)
chart.save("select_by_batch_size.png", scale_factor=2)
